In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\owner\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
# import MinMaxScaler to scale data
from sklearn.preprocessing import MinMaxScaler

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [12]:
# Set features. This will also be used as your x values.
selected_features = df[["koi_disposition","koi_impact","koi_time0bk","koi_period","koi_slogg","koi_depth","koi_prad","koi_insol","koi_model_snr","koi_tce_plnt_num","koi_steff"]]

In [13]:

conf_cand = selected_features[(selected_features['koi_disposition']=='CONFIRMED') | (selected_features['koi_disposition']=='CANDIDATE')]
conf_cand.reset_index(drop=True, inplace=True)

In [14]:
conf_cand

,koi_disposition,koi_impact,koi_time0bk,koi_period,koi_slogg,koi_depth,koi_prad,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff
0,CONFIRMED,0.586,162.513840,54.418383,4.467,874.8,2.83,9.11,25.8,2,5455
1,CONFIRMED,0.701,171.595550,2.525592,4.438,603.3,2.75,926.16,40.9,1,6031
2,CONFIRMED,0.762,172.979370,4.134435,4.486,686.0,2.77,427.65,40.2,2,6046
3,CONFIRMED,0.755,179.554370,2.566589,4.486,226.5,1.59,807.74,15.0,3,6046
4,CONFIRMED,0.052,173.621937,16.068647,4.485,4914.3,5.76,30.75,161.9,1,5031
...,...,...,...,...,...,...,...,...,...,...,...
3482,CANDIDATE,0.218,131.787600,4.736816,4.456,35.3,0.60,395.05,6.9,1,6088
3483,CANDIDATE,0.075,218.271900,130.235324,4.529,750.1,2.44,2.86,9.7,1,5616
3484,CANDIDATE,1.206,137.481093,8.870416,4.027,873.1,39.46,414.26,43.8,1,6022
3485,CANDIDATE,1.230,144.131720,47.109631,3.597,752.2,78.98,75.40,35.1,1,5258


In [15]:
SF_balanced = selected_features.append(conf_cand)

In [16]:
SF_balanced

,koi_disposition,koi_impact,koi_time0bk,koi_period,koi_slogg,koi_depth,koi_prad,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff
0,CONFIRMED,0.586,162.513840,54.418383,4.467,874.8,2.83,9.11,25.8,2,5455
1,FALSE POSITIVE,0.969,175.850252,19.899140,4.544,10829.0,14.60,39.30,76.3,1,5853
2,FALSE POSITIVE,1.276,170.307565,1.736952,4.564,8079.2,33.46,891.96,505.6,1,5805
3,CONFIRMED,0.701,171.595550,2.525592,4.438,603.3,2.75,926.16,40.9,1,6031
4,CONFIRMED,0.762,172.979370,4.134435,4.486,686.0,2.77,427.65,40.2,2,6046
...,...,...,...,...,...,...,...,...,...,...,...
3482,CANDIDATE,0.218,131.787600,4.736816,4.456,35.3,0.60,395.05,6.9,1,6088
3483,CANDIDATE,0.075,218.271900,130.235324,4.529,750.1,2.44,2.86,9.7,1,5616
3484,CANDIDATE,1.206,137.481093,8.870416,4.027,873.1,39.46,414.26,43.8,1,6022
3485,CANDIDATE,1.230,144.131720,47.109631,3.597,752.2,78.98,75.40,35.1,1,5258


# Create a Train Test Split

Use `koi_disposition` for the y values

In [19]:
X = SF_balanced.drop('koi_disposition',axis=1)
y = SF_balanced['koi_disposition']

In [23]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=42)


In [24]:
X_train.head()

,koi_impact,koi_time0bk,koi_period,koi_slogg,koi_depth,koi_prad,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff
5577,0.399,134.052302,6.882686,4.483,373660.0,60.49,169.85,2842.2,1,5704
2907,0.365,132.260680,4.563837,4.496,201.8,1.22,218.18,9.9,1,5408
258,0.164,146.152850,16.568068,4.602,1331.4,2.54,21.27,41.9,1,4987
3881,0.020,157.754400,43.849647,2.644,452.3,21.42,645.68,8.3,1,4893
3397,0.174,143.159880,11.912567,3.893,123.1,2.61,562.27,27.8,1,6337


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [10]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

NameError: name 'model2' is not defined

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)